In [1]:
import numpy as np
from Molecule import Molecule
np.set_printoptions(precision=7, linewidth=120, suppress=True)

In [2]:
mol = Molecule('input/benzene_geom.txt')
mol.print_geom()
H = mol.read_hessian('input/benzene_hessian.txt')
print(H)
masses = {6:12.0000000, 8:15.994914619, 1:1.00782503223}

Number of Atoms : 12
Coordinate File: 
6    0.000000    2.620659    0.000000
6   -2.269558    1.310330    0.000000
6   -2.269558   -1.310330    0.000000
6    0.000000   -2.620659    0.000000
6    2.269558   -1.310330    0.000000
6    2.269558    1.310330    0.000000
1   -4.041307    2.333249    0.000000
1   -4.041307   -2.333249    0.000000
1    0.000000   -4.666499    0.000000
1    4.041307   -2.333249    0.000000
1    4.041307    2.333249    0.000000
1    0.000000    4.666499    0.000000
[[ 0.8370476  0.         0.        ... -0.0814934  0.         0.       ]
 [ 0.         1.0278249  0.        ...  0.        -0.4748941  0.       ]
 [ 0.         0.         0.1884039 ...  0.         0.        -0.0533322]
 ...
 [-0.0814934  0.         0.        ...  0.0769669  0.         0.       ]
 [ 0.        -0.4748941  0.        ...  0.         0.4872162  0.       ]
 [ 0.         0.        -0.0533322 ...  0.         0.         0.0368569]]


In [3]:
atomic_mass_lst = [masses[int(i)] for i in mol.atoms]
print(atomic_mass_lst)

[12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 1.00782503223, 1.00782503223, 1.00782503223, 1.00782503223, 1.00782503223, 1.00782503223]


# Mass-Weight the Hessian Matrix

In [4]:
H_mw = H.copy()

nrows, ncols = H_mw.shape

print(f'Nrows {nrows} NColumns {ncols} in hessian the matrix')

# New atom switches after each three atom as total number of atom is 3 for water 

for i in range(nrows):
    for j in range(ncols):
        denom = np.sqrt(atomic_mass_lst[np.floor_divide(i, 3)] * atomic_mass_lst[np.floor_divide(j, 3)])
        H_mw[i, j] /= denom

print('Mass Weighted Hessian Matrix is : ')
print(H_mw)


Nrows 36 NColumns 36 in hessian the matrix
Mass Weighted Hessian Matrix is : 
[[ 0.069754   0.         0.        ... -0.0234336  0.         0.       ]
 [ 0.         0.0856521  0.        ...  0.        -0.1365569  0.       ]
 [ 0.         0.         0.0157003 ...  0.         0.        -0.0153358]
 ...
 [-0.0234336  0.         0.        ...  0.0763693  0.         0.       ]
 [ 0.        -0.1365569  0.        ...  0.         0.4834333  0.       ]
 [ 0.         0.        -0.0153358 ...  0.         0.         0.0365708]]


# Diagonalize the Mass-Weighted Hessian Matrix

In [5]:
eigenvalues, eigenvector = np.linalg.eigh(H_mw)

print('Eigen values of the Mass weighted Hessian Matrix are')
for i in range(len(eigenvalues)):
    print(f'{i}{eigenvalues[i]:20.10f}')

Eigen values of the Mass weighted Hessian Matrix are
0       -0.0000000000
1       -0.0000000000
2        0.0000000000
3        0.0000000169
4        0.0000000169
5        0.0000000170
6        0.0086494098
7        0.0086494098
8        0.0185522333
9        0.0185522333
10        0.0248951017
11        0.0267799113
12        0.0407588947
13        0.0407588947
14        0.0504228693
15        0.0506351704
16        0.0520195116
17        0.0536487991
18        0.0536487991
19        0.0558373300
20        0.0568687590
21        0.0568687590
22        0.0711678903
23        0.0711678903
24        0.0717564404
25        0.0963092288
26        0.1189063704
27        0.1189063705
28        0.1413237669
29        0.1413237669
30        0.5193179971
31        0.5244972587
32        0.5244972589
33        0.5283614145
34        0.5283614145
35        0.5314353548


# Compute the Harmonic Vibrational Frequencies

In [7]:
from scipy.constants import physical_constants
from numpy.lib.scimath import sqrt as csqrt
c = physical_constants['speed of light in vacuum'][0]
N_A = physical_constants['Avogadro constant'][0]
E_h = physical_constants["Hartree energy"][0]
bohr2m = physical_constants["Bohr radius"][0]

vib_constant = np.sqrt((N_A*E_h*1000)/(bohr2m*bohr2m))/(2*np.pi*c*100)

# print(vib_constant)

# Making any -ve number to postive for the frequency calculation

vib_frequencies = csqrt((eigenvalues)) * vib_constant

for i in vib_frequencies.real:
    print(f'{i:.4f}')


0.0000
0.0000
0.0247
0.6683
0.6683
0.6694
478.0766
478.0766
700.1682
700.1682
811.0754
841.2186
1037.8043
1037.8043
1154.2983
1156.7258
1172.4313
1190.6504
1190.6504
1214.6932
1225.8608
1225.8608
1371.3437
1371.3437
1377.0025
1595.2848
1772.5841
1772.5841
1932.4661
1932.4661
3704.4263
3722.8530
3722.8530
3736.5416
3736.5416
3747.3952
